In [3]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask

# Load Nairobi boundary GeoJSON
nairobi = gpd.read_file("/home/dataopske/Desktop/jav/data/raw/supportdata/nairobi.json")

# Load the raster
raster_path = "/home/dataopske/Desktop/jav/data/raw/worldpop/ken_pop_2025_CN_100m_R2025A_v1.tif"
with rasterio.open(raster_path) as src:
    # Reproject Nairobi GeoJSON to match raster CRS
    nairobi = nairobi.to_crs(src.crs)

    # Clip the raster using reprojected Nairobi geometry
    out_image, out_transform = mask(src, nairobi.geometry, crop=True)
    out_meta = src.meta.copy()

# Update metadata
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Save clipped raster
output_path = "/home/dataopske/Desktop/jav/data/processed/Nairobi_2025_Pop_100m.tif"
with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(out_image)

print("Successfully clipped Nairobi population raster!")

ValueError: Input shapes do not overlap raster.

In [4]:
import geopandas as gpd
import rasterio

nairobi = gpd.read_file("/home/dataopske/Desktop/jav/data/raw/supportdata/nairobi.json")

with rasterio.open("/home/dataopske/Desktop/jav/data/raw/worldpop/ken_pop_2025_CN_100m_R2025A_v1.tif") as src:
    print("Raster CRS:", src.crs)
    print("Raster bounds:", src.bounds)
    print("Nairobi CRS:", nairobi.crs)
    print("Nairobi bounds:", nairobi.total_bounds)


Raster CRS: EPSG:4326
Raster bounds: BoundingBox(left=33.90999914436, bottom=-4.737499645050014, right=41.90666577904, top=5.034166982529986)
Nairobi CRS: EPSG:4326
Nairobi bounds: [36.6509378 -1.444471  37.1038871 -1.1635224]


In [5]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask

# Load and clean Nairobi GeoJSON
nairobi = gpd.read_file("/home/dataopske/Desktop/jav/data/raw/supportdata/nairobi.json")

# Keep only valid geometries
nairobi = nairobi[nairobi.is_valid & ~nairobi.is_empty]

# Dissolve into a single polygon (avoid MultiPolygon confusion)
nairobi = nairobi.dissolve()

# Confirm CRS (should be EPSG:4326)
nairobi = nairobi.to_crs(epsg=4326)

# Open the raster and clip
raster_path = "/home/dataopske/Desktop/jav/data/raw/worldpop/ken_pop_2025_CN_100m_R2025A_v1.tif"

with rasterio.open(raster_path) as src:
    nairobi = nairobi.to_crs(src.crs)
    print("✅ Overlap:", nairobi.intersects(box(*src.bounds)).any())

    # Run mask safely
    out_image, out_transform = mask(src, nairobi.geometry, crop=True)
    out_meta = src.meta.copy()

# Update and save
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

output_path = "/home/dataopske/Desktop/jav/data/processed/Nairobi_2025_Pop_100m.tif"
with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(out_image)

print("✅ Successfully clipped and saved Nairobi population raster!")


NameError: name 'box' is not defined

In [6]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import box  # ✅ add this import

# Load and clean Nairobi GeoJSON
nairobi = gpd.read_file("/home/dataopske/Desktop/jav/data/raw/supportdata/nairobi.json")

# Keep only valid geometries
nairobi = nairobi[nairobi.is_valid & ~nairobi.is_empty]

# Dissolve into a single polygon (avoid MultiPolygon confusion)
nairobi = nairobi.dissolve()

# Confirm CRS (should be EPSG:4326)
nairobi = nairobi.to_crs(epsg=4326)

# Open the raster and clip
raster_path = "/home/dataopske/Desktop/jav/data/raw/worldpop/ken_pop_2025_CN_100m_R2025A_v1.tif"

with rasterio.open(raster_path) as src:
    nairobi = nairobi.to_crs(src.crs)

    # ✅ Check overlap
    print("✅ Overlap:", nairobi.intersects(box(*src.bounds)).any())

    # Clip
    out_image, out_transform = mask(src, nairobi.geometry, crop=True)
    out_meta = src.meta.copy()

# Update and save
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

output_path = "/home/dataopske/Desktop/jav/data/processed/Nairobi_2025_Pop_100m.tif"
with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(out_image)

print("✅ Successfully clipped and saved Nairobi population raster!")


✅ Overlap: True
✅ Successfully clipped and saved Nairobi population raster!


In [ ]:
import rasterio
import numpy as np

# Load your clipped raster
path = "/home/dataopske/Desktop/jav/data/processed/Nairobi_2025_Pop_100m.tif"

with rasterio.open(path) as src:
    data = src.read(1)  # read the first (and only) band
    data = np.where(data < 0, 0, data)  # replace negative values if any
    total_pop = np.nansum(data)

print(f"🧮 Estimated Nairobi population (WorldPop 2025, 100 m): {total_pop:,.0f} people")

🧮 Estimated Nairobi population (WorldPop 2025, 100 m): 5,666,439 people


In [9]:
import folium
import geopandas as gpd
from shapely.geometry import box

# --- Load your Nairobi GeoJSON ---
nairobi_path = "/home/dataopske/Desktop/jav/data/raw/supportdata/nairobi.json"  # adjust path
nairobi_gdf = gpd.read_file(nairobi_path)

# --- Define the raster bounds box ---
raster_bounds = [33.90999914436, -4.737499645050014, 41.90666577904, 5.034166982529986]
raster_box = gpd.GeoDataFrame({"geometry": [box(*raster_bounds)]}, crs="EPSG:4326")

# --- Create the folium map centered roughly on Nairobi ---
center = [nairobi_gdf.geometry.centroid.y.mean(), nairobi_gdf.geometry.centroid.x.mean()]
m = folium.Map(location=center, zoom_start=9, tiles="CartoDB positron")

# --- Add your Nairobi boundary (your JSON file) ---
folium.GeoJson(
    nairobi_gdf,
    name="Your Nairobi boundary",
    style_function=lambda x: {"color": "red", "weight": 3, "fillOpacity": 0.05}
).add_to(m)

# --- Add raster full coverage bounds (Kenya-wide) ---
folium.GeoJson(
    raster_box,
    name="Raster extent (WorldPop)",
    style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.0, "dashArray": "5,5"}
).add_to(m)

# --- Add layer control ---
folium.LayerControl().add_to(m)

m


/tmp/ipykernel_539306/1110290689.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [nairobi_gdf.geometry.centroid.y.mean(), nairobi_gdf.geometry.centroid.x.mean()]


In [15]:
import json
import geopandas as gpd
from shapely.geometry import Point, box
from pyproj import CRS

coord_file = '/home/dataopske/Desktop/jav/data/raw/worldmove/gridcell_coordinates.json'
output_geojson = '/home/dataopske/Desktop/jav/data/raw/worldmove/nairobi_grid_1km.json'

# Load coordinate list
with open(coord_file, 'r') as f:
    coordinates = json.load(f)

# Convert coordinates to Points
points = []
for item in coordinates:
    # Handle cases where item might be nested
    if isinstance(item, dict) and 'lon' in item and 'lat' in item:
        lon, lat = item['lon'], item['lat']
    elif isinstance(item, (list, tuple)) and len(item) >= 2:
        lon, lat = item[0], item[1]
    else:
        continue
    points.append(Point(lon, lat))

# Create GeoDataFrame in WGS84 (lat/lon)
gdf_points = gpd.GeoDataFrame(geometry=points, crs="EPSG:4326")

# Reproject to meters to create 1km x 1km boxes
gdf_meters = gdf_points.to_crs(epsg=32737)  # UTM zone 37S (Kenya)
cell_size = 1000  # 1 km

boxes = []
for point in gdf_meters.geometry:
    x, y = point.x, point.y
    boxes.append(box(x - cell_size/2, y - cell_size/2, x + cell_size/2, y + cell_size/2))

# Convert back to GeoDataFrame and reproject to WGS84
gdf_grid = gpd.GeoDataFrame(geometry=boxes, crs="EPSG:32737").to_crs(epsg=4326)

# Save as GeoJSON
gdf_grid.to_file(output_geojson, driver='GeoJSON')
print(f"✅ Grid GeoJSON saved to: {output_geojson}")
print(f"📦 Total grid cells: {len(gdf_grid)}")


✅ Grid GeoJSON saved to: /home/dataopske/Desktop/jav/data/raw/worldmove/nairobi_grid_1km.json
📦 Total grid cells: 0


In [17]:
import rasterio
from rasterio.mask import mask

with rasterio.open('/home/dataopske/Desktop/jav/data/raw/worldpop/ken_pop_2025_CN_100m_R2025A_v1.tif') as src:
    grid_gdf = gpd.read_file(output_geojson)
    out_image, out_transform = mask(src, grid_gdf.geometry, crop=False)


ValueError: min() iterable argument is empty

In [18]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
import pandas as pd
from tqdm import tqdm

# Paths
raster_path = "/home/dataopske/Desktop/jav/data/processed/Nairobi_2025_Pop_100m.tif"
grid_path = "/home/dataopske/Desktop/jav/data/raw/worldmove/nairobi_grid_1km.json"

# Load grid polygons
grid_gdf = gpd.read_file(grid_path)

# Read raster and match CRS
with rasterio.open(raster_path) as src:
    grid_gdf = grid_gdf.to_crs(src.crs)
    
    pop_estimates = []
    for i, row in tqdm(grid_gdf.iterrows(), total=len(grid_gdf)):
        geom = [row.geometry]
        
        try:
            # Clip raster to grid cell
            out_image, _ = mask(src, geom, crop=True)
            data = out_image[0]
            
            # Clean up data and sum
            data = np.where(data < 0, 0, data)
            total_pop = np.nansum(data)
        except Exception:
            total_pop = np.nan
        
        pop_estimates.append(total_pop)

# Combine results
grid_gdf["pop_estimate"] = pop_estimates

# Export for further use
output_csv = "/home/dataopske/Desktop/jav/data/processed/nairobi_grid_population_2025.csv"
grid_gdf[["geometry", "pop_estimate"]].to_file(
    "/home/dataopske/Desktop/jav/data/processed/nairobi_grid_population_2025.geojson", driver="GeoJSON"
)
grid_gdf[["pop_estimate"]].to_csv(output_csv, index=False)

print(f"✅ Done! Saved population estimates for {len(grid_gdf)} grid cells.")
print(f"📊 Total estimated Nairobi population (sum of cells): {grid_gdf['pop_estimate'].sum():,.0f} people")


0it [00:00, ?it/s]

✅ Done! Saved population estimates for 0 grid cells.
📊 Total estimated Nairobi population (sum of cells): 0 people


In [21]:
import json
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np

coord_file = '/home/dataopske/Desktop/jav/data/raw/worldmove/gridcell_coordinates.json'

with open(coord_file, 'r') as f:
    coords = json.load(f)

cells = []
half_km_deg = 0.009  # ~1 km in degrees at the equator (approx)

for i, item in enumerate(coords):
    # Handle both list [lon, lat] or dict {"lon": ..., "lat": ...}
    if isinstance(item, dict):
        lon, lat = item.get("lon"), item.get("lat")
    else:
        lon, lat = item
    
    if lon is None or lat is None:
        continue

    poly = Polygon([
        (lon - half_km_deg, lat - half_km_deg),
        (lon + half_km_deg, lat - half_km_deg),
        (lon + half_km_deg, lat + half_km_deg),
        (lon - half_km_deg, lat + half_km_deg)
    ])
    cells.append({"cell_id": i, "geometry": poly})

grid_gdf = gpd.GeoDataFrame(cells, crs="EPSG:4326")

# Save properly
output_path = "/home/dataopske/Desktop/jav/data/processed/nairobi_grid_1km.geojson"
grid_gdf.to_file(output_path, driver="GeoJSON")

print(f"✅ Created valid grid file with {len(grid_gdf)} polygons.")
print(f"🗺️  Bounds: {grid_gdf.total_bounds}")


ValueError: not enough values to unpack (expected 2, got 1)

In [19]:
import geopandas as gpd
import rasterio

grid = gpd.read_file("/home/dataopske/Desktop/jav/data/raw/worldmove/nairobi_grid_1km.json")

with rasterio.open("/home/dataopske/Desktop/jav/data/processed/Nairobi_2025_Pop_100m.tif") as src:
    print("Raster CRS:", src.crs)
    print("Raster bounds:", src.bounds)

print("Grid CRS:", grid.crs)
print("Grid bounds:", grid.total_bounds)


Raster CRS: EPSG:4326
Raster bounds: BoundingBox(left=36.65083246673, bottom=-1.4449996582200142, right=37.10416579825, top=-1.1633329926800142)
Grid CRS: EPSG:4326
Grid bounds: [nan nan nan nan]


In [20]:
import geopandas as gpd
from shapely.geometry import Point, box
import json
import pandas as pd

coord_file = "/home/dataopske/Desktop/jav/data/raw/worldmove/gridcell_coordinates.json"
with open(coord_file) as f:
    coords = json.load(f)

rows = []
for i, c in enumerate(coords):
    # Handle either dicts or lists
    if isinstance(c, dict) and 'lat' in c and 'lon' in c:
        lon, lat = c['lon'], c['lat']
    elif isinstance(c, (list, tuple)) and len(c) == 2:
        lon, lat = c
    else:
        continue
    rows.append({"cell_id": i, "lon": lon, "lat": lat})

df = pd.DataFrame(rows)

# Build 1 km bounding boxes (~0.009° ≈ 1 km)
size_deg = 0.009
df["geometry"] = df.apply(lambda r: box(
    r["lon"] - size_deg / 2, r["lat"] - size_deg / 2,
    r["lon"] + size_deg / 2, r["lat"] + size_deg / 2
), axis=1)

grid_gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

# Save new valid GeoJSON
output = "/home/dataopske/Desktop/jav/data/processed/nairobi_grid_1km.json"
grid_gdf.to_file(output, driver="GeoJSON")

print(f"✅ Created {len(grid_gdf)} 1 km grid polygons and saved to:")
print(output)
print("Bounds:", grid_gdf.total_bounds)


ValueError: Cannot set a DataFrame without columns to the column geometry